In [1]:
import altair as alt

# Análisis exploratorio visual del SARS-CoV-2

En este ejercicio, vamos a aplicar algunos de los conocimientos que hemos ido adquiriendo durante el curso para 
visualizar el contenido en nucleótidos de unas cepas de coronavirus recientemente secuenciadas. 

En una primera parte, vamos a calcular cómo varía un k-mero a lo largo de cada uno de los genomas para después compararlos visualmente. 

En una segunda parte, vamos a extraer los k-meros más frecuentes y los visualizaremos por frecuencias y rangos en diagramas de barras para establecer diferencias y coincidencias entre las 4 secuencias. 

Primero, vamos a cargar el fasta con las secuencias para las 4 cepas de COVID:

In [2]:
fd = open("../data/covid-samples.fasta")

In [3]:
covid_seqs = fd.readlines()

In [4]:
i = 0
seqs = []
while(i < len(covid_seqs)):
    seq_name = covid_seqs[i].strip(">").strip("\n")
    seq = ''
    i+=1
    while(i < len(covid_seqs) and covid_seqs[i][0] != '>'):
        seq += covid_seqs[i].strip('\n')
        i+=1
    seq = seq.upper()
    seqs.append({"seq_name": seq_name, "seq_short_name": seq_name[:11], "seq": seq})

In [5]:
seqs

[{'seq_name': 'MW186669.1 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/EGY/Cairo-sample 19 MOH/2020, complete genome',
  'seq_short_name': 'MW186669.1 ',
  'seq': '...'},
 {'seq_name': 'MW186829.1 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/EGY/Cairo-sample 2 MOH/2020 ORF1ab polyprotein (ORF1ab), ORF1a polyprotein (ORF1ab), surface glycoprotein (S), ORF3a protein (ORF3a), envelope protein (E), membrane glycoprotein (M), ORF6 protein (ORF6), ORF7a protein (ORF7a), ORF7b (ORF7b), ORF8 protein (ORF8), nucleocapsid phosphoprotein (N), and ORF10 protein (ORF10) genes, complete cds',
  'seq_short_name': 'MW186829.1 ',
  'seq': '...'},
 {'seq_name': 'MW186830.1 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/EGY/Cairo-sample 8 MOH/2020 ORF1ab polyprotein (ORF1ab), ORF1a polyprotein (ORF1ab), surface glycoprotein (S), ORF3a protein (ORF3a), envelope protein (E), membrane glycoprotein (M), ORF6 protein (ORF6), 

In [6]:
for seq_name in [seq['seq_name'] for seq in seqs]:
    print(seq_name)
    print()

MW186669.1 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/EGY/Cairo-sample 19 MOH/2020, complete genome

MW186829.1 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/EGY/Cairo-sample 2 MOH/2020 ORF1ab polyprotein (ORF1ab), ORF1a polyprotein (ORF1ab), surface glycoprotein (S), ORF3a protein (ORF3a), envelope protein (E), membrane glycoprotein (M), ORF6 protein (ORF6), ORF7a protein (ORF7a), ORF7b (ORF7b), ORF8 protein (ORF8), nucleocapsid phosphoprotein (N), and ORF10 protein (ORF10) genes, complete cds

MW186830.1 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/EGY/Cairo-sample 8 MOH/2020 ORF1ab polyprotein (ORF1ab), ORF1a polyprotein (ORF1ab), surface glycoprotein (S), ORF3a protein (ORF3a), envelope protein (E), membrane glycoprotein (M), ORF6 protein (ORF6), ORF7a protein (ORF7a), ORF7b (ORF7b), ORF8 protein (ORF8), nucleocapsid phosphoprotein (N), and ORF10 protein (ORF10) genes, complete cds

MW181431.1 |

## Problema 1: Fluctuación de codones
Ahora, vamos a ver cómo fluctúan ciertos k-mero (p. ej. codones) en cada una de las cepas. 
Para ello, vamos a resolver el siguiente problema:

Dado un genoma G de longitud n, se da la necesidad de saber cómo fluctúan determinados nucleótidos (extensible a k-meros) a través del mismo. Para calcularlo, se quieren tener en cuenta los siguientes parámetros:

- w: tamaño de la ventana (window) en la que se cuentan los k-meros
- kmer: secuencia a buscar, de longitud menor que w
- s: paso o salto (step) para avanzar a la siguiente ventana

s y w permiten modular la ‘resolución’ de la búsqueda, y también la velocidad del algoritmo.

Por lo tanto, vamos a definir la función `cuenta_kmero_ventana(seq_obj, w=1000, s=250, kmero='TGA')`, que busque el k-mero que se le pasa (búsqueda simple en cada ventana), y devuelva una lista de diccionarios con el siguiente formato:

```
{
    "i": "ventana #i",
    "count": "frecuencia absoluta del kmero en la ventana #i"
}
```

1. El k-mero a buscar por defecto será el codón de inicio `"AUG"` (`"ATG"`).
2. Usando Altair, y siguiendo lo que vimos en el lab6-b, dibuja la fluctuación de este codón en cada una de las secuencias.
3. Luego intenta pintarlas todas juntas usando el operador de concatenación "&".

In [7]:
def cuenta_kmero_ventana(seq_obj, w=1000, s=250, kmero='ATG'):
    i = 0
    freqs = []
    while i < len(seq_obj['seq']) - len(kmero) + 1:
        freqs.append({"i": i, "count": seq_obj['seq'][i:i+w].count(kmero)})
        i+=s
    return freqs

In [8]:
freqs = [{
    "seq_name": seqs[i]['seq_name'], 
    "freq": cuenta_kmero_ventana(seqs[i], w=1000, s=250, kmero='ATG')} for i in range(len(seqs))]

In [9]:
a  = alt.Chart(alt.Data(values=freqs[0]['freq'])).mark_line().encode(x="i:Q", y="count:Q")
b = alt.Chart(alt.Data(values=freqs[0]['freq'])).mark_line().encode(x="i:Q", y="count:Q")

In [10]:
a & b

alt.VConcatChart(...)

In [11]:
alt.Chart(alt.Data(values=freqs[1]['freq'])).mark_line().encode(x="i:Q", y="count:Q").interactive()

alt.Chart(...)

In [12]:
alt.Chart(alt.Data(values=freqs[2]['freq'])).mark_line().encode(x="i:Q", y="count:Q").interactive()

alt.Chart(...)

In [13]:
alt.Chart(alt.Data(values=freqs[3]['freq'])).mark_line().encode(x="i:Q", y="count:Q").interactive()

alt.Chart(...)

### Visualizando las 4 secuencias juntas para detectar diferencias

Para visualizar los datos en altair, tendremos que normalizarlos: esto es, cada entrada en nuestra lista tendrá que tener el siguiente formato:

```
{
    "seq_short_name": "nombre (acortado) de la secuencia a la que se refiere este dato", 
    "count": "frecuencia en la ventana #i", 
    "i": "ventana #i"
}
```

In [14]:
freqs_normalized = [
    {"seq_short_name": seq_obj['seq_name'][:11], 
     "count": point['count'], 
     "i": point['i']} for seq_obj in freqs for point in seq_obj['freq']]

In [15]:
alt.Chart(alt.Data(
    values=freqs_normalized
)).mark_line().encode(
    x="i:Q", 
    y="count:Q", 
    color="seq_short_name:N").interactive()

alt.Chart(...)

## Problema 2: Visualizando secuencias frecuentes en el SARS-CoV-2

Vamos a crear una visualización para comparar los n k-meros más comunes encontrados en las distintas cepas.

Primero, vamos a ver algunos resultados en "crudo": 

In [16]:
def kmeros_frecuentes(secuencia, k, descarta_errores=False):
    freq = {}
    n = len(secuencia)
    for i in range(n-k+1):
        kmero = secuencia[i:i+k]
        if kmero in freq:
            freq[kmero] += 1
        else:
            freq[kmero] = 1
    
    if descarta_errores and 'N'*k in freq:
        del freq['N'*k]
    return freq

In [17]:
def top_kmeros(secuencia, k, descarta_errores=False):
    kmeros = []
    freqs = kmeros_frecuentes(secuencia, k, descarta_errores)
    m = max(freqs.values())
    for key in freqs:
        if freqs[key] == m:
            kmeros.append(key)
        # add each key to words whose corresponding frequency value is equal to m
    return kmeros, m

In [18]:
for seq in seqs:
    print(top_kmeros(seq['seq'], 9))

(['TAAACGAAC'], 7)
(['NNNNNNNNN'], 1275)
(['NNNNNNNNN'], 796)
(['NNNNNNNNN'], 443)


In [19]:
for seq in seqs:
    print(top_kmeros(seq['seq'], 9, descarta_errores=True))

(['TAAACGAAC'], 7)
(['TAAACGAAC'], 7)
(['TAAACGAAC'], 6)
(['AAAAAAAAA'], 25)


### Top n k-meros
Como podemos detectar algunas coincidencias, vamos a intentar visualizarlas. Para ello, primero definiremos una 
función `top_n_kmeros(seq, k, n)` que devolverá los n k-meros más frecuentes en la secuencias haciendo uso de la función `kmeros_frecuentes`. 

Después, declararemos otra función `top_n_kmeros_barchart(seqs, k=3, top_n=10)` en la que visualizaremos los n (10) 3 meros más frecuentes en cada una de las secuencias.  

In [20]:
def top_n_kmeros(seq, k, n):
    kmeros = []
    freqs = kmeros_frecuentes(seq, k, descarta_errores=True)
#     print(freqs)
    min_val = sorted(list(set(freqs.values())))[-n]
    for key in freqs:
        if freqs[key] >= min_val:
            kmeros.append({"kmer": key, "freq": freqs[key]})
        # add each key to words whose corresponding frequency value is equal to m
#     return sorted(kmeros, key=lambda x: x['freq'], reverse=True)
    return kmeros

In [21]:
def top_n_kmeros_barchart(seqs, k, top_n):    
    vis_data = []
    for seq in seqs:
        top_n_seq = top_n_kmeros(seq['seq'], k, top_n)
        for kmer_freqs_obj in top_n_seq:
            vis_data.append({
                "seq_short_name": seq['seq_short_name'],
                "kmer": kmer_freqs_obj['kmer'],
                "freq": kmer_freqs_obj['freq'],
            })

    return alt.Chart(alt.Data(values=vis_data)).mark_bar().encode(
        x=alt.X('kmer:N', sort=alt.EncodingSortField(field="freq", op="sum", order='descending')),
        y='freq:Q',
        color='seq_short_name:N',
        tooltip=['seq_short_name:N', 'freq:Q']
    )

In [22]:
top_n_kmeros_barchart(seqs, k=4, top_n=10)

alt.Chart(...)